## GOAL OF THIS NB:

MDB has several utility tables that contain static information. This includes names and ids, for:
   
 * Dungeons
 * Affixes
 * Specs
 * Periods (old, non-current)
 
 This information gets updated very infrequently (once every 2 years). This notebook load this static information into MDB.
 
### Assumptions:
 
We assume the tables already exist in MDB. They should have been created via the script (sql_script/schema.sql). If the table is missing, I will create it manually, and add to script later.

In [63]:
import blizz_api
import blizz_credentials
import blizz_parser
import utils
import mplusdb

import importlib
import pandas as pd

pd.options.display.max_rows = 999

In [20]:
# this token expires every 24 hours
auth = blizz_credentials.Credentials("config/blizzard_api_access.ini")
auth.access_token

'US8GxvFQtnTshzCjvOuD1qKQ3yvsH5tx9b'

In [72]:
# initialize MDB object for uploading data
importlib.reload(mplusdb)
mdb = mplusdb.MplusDatabase(config_file_path="config/db_config.ini")

In [33]:
### USE THIS CELL TO REMIND YOURSELF WHAT METHODS TO CALL

# fetch mythic+ period index
#get_period_startend(self, region:str, period:int) -> Tuple[int, int]
#importlib.reload(utils)
#importlib.reload(blizz_api)
#importlib.reload(blizz_parser)
#caller = blizz_api.Caller(access_token=auth.access_token)
#ret = caller.get_spec_by_id(577)
#ret = caller.get_period_startend(region="us", period=641)
#ret = caller.get_leaderboard(region="us", dungeon=382,period=780, realm=11)
#print(ret)

---
#### PERIOD TABLE

In [57]:
# Go through each period (641 => 792)
# and get timestamps and affix info
period_data = []

caller = blizz_api.Caller(access_token=auth.access_token)
for period in range(641, 792+1):
    # I don't know the exact period border between the expansions,
    # so try 3 base dungeons from each. One should post, two others will error.
    possible_dungeons = [197, 244, 382] # a legion / bfa / sl dungeon
    affixes = None
    for dungeon in possible_dungeons:
        try:
            leaderboard = caller.get_leaderboard(region="us", dungeon=dungeon, period=period, realm=11)
            affixes = leaderboard.affixes
        except:
            pass
    is_tyrannical = 1 if 9 in affixes else 0
    # get regional period start & end
    for region in ["us", "eu", "tw", "kr"]:
        start, end = caller.get_period_startend(region=region, period=period)
        affixes_token = "_".join([str(affix_id) for affix_id in affixes])
        period_data.append((region, period, start, end, is_tyrannical, affixes_token))

In [78]:
# replace region string tokens with ints
period_data_edit = []
region = {"us":1, "eu":3, "kr":2, "tw":4}
for row in period_data:
    new_row = list(row)
    new_row[0] = region[row[0]]
    period_data_edit.append(new_row)

In [79]:
# dump into MDB
mdb.insert(table="period", data=period_data_edit)

**NOTE:**

Period endpoint only goes back to 641. Leaderboard goes back to 597. At some point, I need to go back to 597, and get that data via leaderboard endpoint.

---
#### AFFIX TABLE

In [ ]:
"""
+-----------------+-----------------+------+-----+---------+-------+
| Field           | Type            | Null | Key | Default | Extra |
+-----------------+-----------------+------+-----+---------+-------+
| region          | int unsigned    | NO   | PRI | NULL    |       |
| id              | int unsigned    | NO   | PRI | NULL    |       |
| start_timestamp | bigint unsigned | NO   |     | NULL    |       |
| end_timestamp   | bigint unsigned | NO   |     | NULL    |       |
| tyrannical      | tinyint(1)      | YES  |     | 0       |       |
| affixes         | varchar(50)     | YES  |     | NULL    |       |
+-----------------+-----------------+------+-----+---------+-------+
"""